In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Primero guardamos las coordenadas del centro de masa para cada tiempo

### Gadget

In [10]:
import matplotlib.pyplot as plt
import h5py
import rotation as rot
import sphviewer as sph
import numpy as np
import time_conversion as tiempo

vector = ('00','01','03')
vector2 = ('M31', 'MW', 'M33')
h=0.732
G = 4.299e-6
a0=1
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter

# path = '/home/ornela/SimCLUES/'
path = '/home/omarioni/'
atime = np.loadtxt(path + 'redshift_outputs.txt')
aexp = atime[:,2]
aexp = aexp[::-1]

path = '/mnt/is2/alejandro/ornella/'

snapshot=range(496,495,-1)

for isnap in snapshot:
    snap = h5py.File(path + 'outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')
    
    for i in range(0,3):
        cm   = snap['subhalo_0'+ str('%s'%vector[i])+ '/Center'].value
        r200 = snap['subhalo_0'+ str('%s'%vector[i])+ '/R200'].value
        
        pstr  = snap['subhalo_0'+ str('%s'%vector[i]) + '/Str/Coordinates'].value
        mstr  = snap['subhalo_0'+ str('%s'%vector[i]) + '/Str/Masses'].value
        vel   = snap['subhalo_0'+ str('%s'%vector[i])+ '/Str/Velocities'].value
        IDs   = snap['subhalo_0'+ str('%s'%vector[i])+ '/Str/Ids'].value
        tform = snap['subhalo_0'+ str('%s'%vector[i])+ '/Str/FormationTime'].value
        
        z = a0/tform - 1.
        Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
        tform = tiempo.conv(z, h, omega_lambda, omega_matter)  
        
#---aca paso las coordenadas respecto al centro de la galaxia------
        xstr = (pstr[:,0]-cm[0])*aexp[0]/h
        ystr = (pstr[:,1]-cm[1])*aexp[0]/h
        zstr = (pstr[:,2]-cm[2])*aexp[0]/h
        rstr = np.sqrt(xstr**2+ystr**2+zstr**2)
        
        r200 = r200*aexp[0]/h
        
        rgal=0.15*r200
        
        limit, = np.where(rstr<rgal)
        
        rn = rstr[limit]
        ID = IDs[limit]
        tf = tform[limit]
        
#----------------------------------------------------------------------------------

        data = np.zeros([np.size(rn),3])
        data[:,0]= ID
        data[:,1]= rn
        data[:,2]= tf
        
        
        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_tform_z0.dat', data, fmt='%12.6f')

In [7]:
import matplotlib.pyplot as plt
import h5py
import rotation as rot
import sphviewer as sph
import numpy as np
import time_conversion as tiempo

vector = ('00','01','03')
vector2 = ('M31', 'MW', 'M33')
h=0.732
G = 4.299e-6
a0=1
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter

# path = '/home/ornela/SimCLUES/'
path = '/home/omarioni/'
atime = np.loadtxt(path + 'redshift_outputs.txt')
aexp = atime[:,2]

path = '/mnt/is2/alejandro/ornella/'

snapshot=range(496,0,-1)

for isnap in snapshot:
    snap = h5py.File(path + 'outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')
    
    for i in range(0,3):
        cm   = snap['subhalo_0'+ str('%s'%vector[i])+ '/Center'].value
         
        z = a0/aexp[isnap] - 1.
        Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
        time = tiempo.conv(z, h, omega_lambda, omega_matter)  

#----------------------------------------------------------------------------------

        path2 = '/home/omarioni/Barras_GdGs/Barras_Gd/_data/'
        archivo1 =  open(path2 + str('%s'%vector2[i])+'_masscenter.dat','a')
        archivo1.write(str('%12.6f'% time) +'\t'+
                str('%12.6f'% cm[0]) +'\t'+
                str('%12.6f'% cm[1]) +'\t'+
                str('%12.6f'% cm[2]) +'\n')
        archivo1.close()